In [1]:
import pandas as pd
import sqlite3
from bs4 import BeautifulSoup
import re

In [5]:
df = pd.read_csv('result.tar.gz', compression='gzip', sep='\t', error_bad_lines=False)

b'Skipping line 37039: expected 12 fields, saw 15\nSkipping line 45611: expected 12 fields, saw 15\n'
b'Skipping line 354339: expected 12 fields, saw 18\nSkipping line 360804: expected 12 fields, saw 102\nSkipping line 360819: expected 12 fields, saw 78\nSkipping line 379126: expected 12 fields, saw 14\nSkipping line 444446: expected 12 fields, saw 16\nSkipping line 449079: expected 12 fields, saw 16\nSkipping line 450250: expected 12 fields, saw 15\n'
b'Skipping line 481374: expected 12 fields, saw 18\nSkipping line 510619: expected 12 fields, saw 20\nSkipping line 512271: expected 12 fields, saw 14\nSkipping line 514317: expected 12 fields, saw 15\nSkipping line 519707: expected 12 fields, saw 40\n'
b'Skipping line 538495: expected 12 fields, saw 13\n'
b'Skipping line 765901: expected 12 fields, saw 56\nSkipping line 773310: expected 12 fields, saw 43\n'
b'Skipping line 805899: expected 12 fields, saw 13\nSkipping line 868492: expected 12 fields, saw 21\nSkipping line 871801: expecte

In [8]:
df.head()

,result.tsv,url,video_url,odd,tip_name,author_name,bookmaker_name,source_name,match_date_time,league_name_in_russian,team_of_season_1_name_in_russian,team_of_season_2_name_in_russian
0,Болгария. София принимает Витошу. Ну здесь вро...,https://vprognoze.ru/forecast/newbie/fcufootba...,NaN,4.8,NaN,dobruijin,1xСтавка,Vprognoze.ru,2019-02-25 15:30:00,Первая лига,Септември София,Витоша Бистрица
1,Тридцатого марта ожидается внушительная порция...,https://sportivnye-prognozy.ru/football/lejpts...,NaN,1.76,NaN,Sportivnye-prognozy.ru,Winline,Sportivnye-prognozy.ru,2019-03-30 17:30:00,Бундеслига,Лейпциг,Герта
2,Встречаются лидеры второй лиги и одна из лучши...,https://www.vseprosport.ru,NaN,0,NaN,Андрей Бондаренко,1xСтавка,Всё про спорт.ру,2019-04-02 16:30:00,Кубок,Падерборн,Гамбург
3,"Матч, который будет иметь большое значение для...",https://bookmaker-ratings.ru/tips/liverpul-che...,NaN,0,NaN,Алексей Андронов,Лига Ставок,Рейтинг Букмекеров,2019-04-14 15:30:00,Премьер-лига,Ливерпуль,Челси
4,"Здесь, пожалуй, все ясно. ""Ливерпуль"" победил ...",https://bookmaker-ratings.ru/tips/portu-liverp...,NaN,0,NaN,Константин Генич,Winline,Рейтинг Букмекеров,2019-04-17 19:00:00,Лига чемпионов УЕФА,Порту,Ливерпуль


In [9]:
df.tail()

,result.tsv,url,video_url,odd,tip_name,author_name,bookmaker_name,source_name,match_date_time,league_name_in_russian,team_of_season_1_name_in_russian,team_of_season_2_name_in_russian
870358,Чемпионат мира по баскетболу откроет матч межд...,https://metaratings.ru/prognozy/basketbol/ango...,NaN,1.89,"Янник Морейра ТБ10,5",Metaratings,Марафон,Metaratings,2019-08-31 07:30:00,Чемпионат Мира,Ангола,Сербия
870359,В очередном матче молодежного чемпионата мира ...,https://metaratings.ru/prognozy/futbol/yaponiy...,NaN,1.78,Япония выйдет в 1/4 финала,Metaratings,Марафон,Metaratings,2019-06-04 15:30:00,Чемпионат мира U20,Япония U20,Южная Корея U20
870360,"Япония и Чили решат кому три очка, может и под...",https://bookmaker-ratings.ru/tips/yaponiya-chi...,NaN,1.96,Япония забьет,Денис Павлий,Пари-Матч,Рейтинг Букмекеров,2019-06-17 23:00:00,Кубок Америки,Япония,Чили
870361,Япония — Тринидад и Тобаго: прогноз и ставка н...,https://betonmobile.ru/prognoz-i-stavka-na-mat...,NaN,1.74,Япония победит и не пропустит,Роман Васильев,Пари-Матч,BetON,2019-06-05 10:30:00,Международные товарищеские матчи,Япония,Тринидад и Тобаго
870362,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [10]:
print(df.shape)

(870363, 12)


In [11]:
print(df.columns)

Index(['result.tsv', 'url', 'video_url', 'odd', 'tip_name', 'author_name',
       'bookmaker_name', 'source_name', 'match_date_time',
       'league_name_in_russian', 'team_of_season_1_name_in_russian',
       'team_of_season_2_name_in_russian'],
      dtype='object')


In [12]:
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 870363 entries, 0 to 870362
Data columns (total 12 columns):
result.tsv                          870362 non-null object
url                                 705051 non-null object
video_url                           1974 non-null object
odd                                 705037 non-null object
tip_name                            705025 non-null object
author_name                         704954 non-null object
bookmaker_name                      704955 non-null object
source_name                         704955 non-null object
match_date_time                     704955 non-null object
league_name_in_russian              704955 non-null object
team_of_season_1_name_in_russian    704955 non-null object
team_of_season_2_name_in_russian    704955 non-null object
dtypes: object(12)
memory usage: 79.7+ MB
None


In [13]:
df.describe()

,result.tsv,url,video_url,odd,tip_name,author_name,bookmaker_name,source_name,match_date_time,league_name_in_russian,team_of_season_1_name_in_russian,team_of_season_2_name_in_russian
count,870362,705051,1974,705037.0,705025,704954,704955,704955,704955,704955,704955,704955
unique,628231,552436,1422,1932.0,14995,77358,34,32,7128,290,3803,3760
top,-,https://www.vseprosport.ru,Вильярреал,1.7,П1,betfaq.ru,1xСтавка,Ставка TV,2019-04-11 19:00:00,Премьер-лига,Ливерпуль,Ливерпуль
freq,93568,3928,9,6235.0,94642,4429,213678,192310,3691,82392,10936,12809


In [18]:
df.sort_values("match_date_time", ascending = False).head(10)

,result.tsv,url,video_url,odd,tip_name,author_name,bookmaker_name,source_name,match_date_time,league_name_in_russian,team_of_season_1_name_in_russian,team_of_season_2_name_in_russian
399166,"Я практически ничего не знаю про Чайку,лишь кр...",https://vprognoze.ru/forecast/newbie/fcufootba...,NaN,2.27,П1,shearer09,1xСтавка,Vprognoze.ru,2019-10-29 15:00:00,ФНЛ,Чайка,Факел
635454,В первом туре чемпионата ФНЛ Факел уступил мос...,https://betzona.ru/fakel-chayka-peschanokopsko...,NaN,1.61,ТМ(2.5),atm1903,1xСтавка,betzona.ru,2019-10-29 15:00:00,ФНЛ,Чайка,Факел
168907,Чемпионат России ФНЛ чайка едет в гости к Факе...,https://vprognoze.ru/forecast/newbie/fcufootba...,NaN,1.6,Двойной шанс X2,Guendouziq,1xСтавка,Vprognoze.ru,2019-10-29 15:00:00,ФНЛ,Чайка,Факел
399087,Не смотря о заявлении тренера о плохой физичес...,https://vprognoze.ru/forecast/newbie/fcufootba...,NaN,2.21,П1,balalaika,1xСтавка,Vprognoze.ru,2019-10-29 15:00:00,ФНЛ,Чайка,Факел
836999,"Доброго времени суток, господа. Воронеж доволь...",https://vprognoze.ru/forecast/newbie/fcufootba...,NaN,1.55,ФОРА1 (0),Денский,1xСтавка,Vprognoze.ru,2019-10-29 15:00:00,ФНЛ,Чайка,Факел
399249,Видео в селе факел не видел чайку Но одно могу...,https://betting.team/ru/predictions/9289636,NaN,2.29,П1,Владимир Барашкин,Winline,Betting Insider,2019-10-29 15:00:00,ФНЛ,Чайка,Факел
399218,"По нашему прогнозу, у «Чайки» есть неплохие пе...",https://www.vseprosport.ru,NaN,2.25,П1,Михаил Савин,BETCITY,Всё про спорт.ру,2019-10-29 15:00:00,ФНЛ,Чайка,Факел
433259,Что ж тут сказатььне мог пройтий меня интрига ...,https://kushvsporte.ru/stavka/1658925-Barselon...,NaN,1.8,П1,Astanabettingbettor,1xСтавка,KushVsporte.ru,2019-10-26 11:00:00,Примера,Барселона,Реал Мадрид
859123,Считаю в не все так просто в данномьпротивосто...,https://kushvsporte.ru/stavka/1658925-Barselon...,NaN,3.9,Х,Astanabettingbettor,1xСтавка,KushVsporte.ru,2019-10-26 11:00:00,Примера,Барселона,Реал Мадрид
703402,Вот такой ординар так скать будет выяснения кт...,https://kushvsporte.ru/stavka/1658925-Barselon...,NaN,2.85,Ф1 (-1.5),Astanabettingbettor,1xСтавка,KushVsporte.ru,2019-10-26 11:00:00,Примера,Барселона,Реал Мадрид


In [19]:
df.sort_values("match_date_time", ascending = True).head(10)

,result.tsv,url,video_url,odd,tip_name,author_name,bookmaker_name,source_name,match_date_time,league_name_in_russian,team_of_season_1_name_in_russian,team_of_season_2_name_in_russian
859646,Команда которая обязана побеждать в этом матче,https://betting.team/ru/predictions/5058054,NaN,1.27,Х2,100% ВЫЙГРЫШ.,Марафон,Betting Insider,2018-12-24 17:00:00,Суперлига,Антальяспор,Фенербахче
859647,Мой прогноз,https://betting.team/ru/predictions/5054522,NaN,1.25,Х2,Homesporttime Sport,Марафон,Betting Insider,2018-12-24 17:00:00,Суперлига,Антальяспор,Фенербахче
290263,Чуйка,https://betting.team/ru/predictions/5061514,NaN,2.22,ОЗ - Нет,Рома Луцик,Марафон,Betting Insider,2018-12-24 17:00:00,Суперлига,Антальяспор,Фенербахче
17734,Выиграют,https://betting.team/ru/predictions/5054914,NaN,2.03,1Х,Lex Lexusov,Марафон,Betting Insider,2018-12-24 17:00:00,Суперлига,Антальяспор,Фенербахче
317835,"Всем, привет! Снова наткнулся на странный матч...",https://vprognoze.ru/forecast/newbie/fcufootba...,NaN,4.05,П1,lcf48,1xСтавка,Vprognoze.ru,2018-12-24 17:00:00,Суперлига,Антальяспор,Фенербахче
317836,Не понимаю в чем тут Фенербахче фавориты поэто...,https://vprognoze.ru/forecast/newbie/fcufootba...,NaN,3.86,П1,FootbalPro,1xСтавка,Vprognoze.ru,2018-12-24 17:00:00,Суперлига,Антальяспор,Фенербахче
17733,Просто я не знаю,https://betting.team/ru/predictions/5055319,NaN,2.03,1Х,Rufat Mirzoyev,Марафон,Betting Insider,2018-12-24 17:00:00,Суперлига,Антальяспор,Фенербахче
17732,Должна сыграть,https://betting.team/ru/predictions/5057675,NaN,1.98,1Х,Макс Зозуляк,Марафон,Betting Insider,2018-12-24 17:00:00,Суперлига,Антальяспор,Фенербахче
17731,Фенербахче не в лучшей форме.Уже месяц они не ...,https://betting.team/ru/predictions/5057875,NaN,1.98,1Х,Рамазан Валеев,Марафон,Betting Insider,2018-12-24 17:00:00,Суперлига,Антальяспор,Фенербахче
271835,Хоть и фен в плохой форме но 1 гол я думаю заб...,https://betting.team/ru/predictions/5059337,NaN,1.69,ОЗ - Да,Илюша Сергеевич,Марафон,Betting Insider,2018-12-24 17:00:00,Суперлига,Антальяспор,Фенербахче


In [16]:
df.count()

result.tsv                          870362
url                                 705051
video_url                             1974
odd                                 705037
tip_name                            705025
author_name                         704954
bookmaker_name                      704955
source_name                         704955
match_date_time                     704955
league_name_in_russian              704955
team_of_season_1_name_in_russian    704955
team_of_season_2_name_in_russian    704955
dtype: int64